# Redis

## Redis Basics

* Redis is a self-described "data structure store" that's in-memory and single-threaded
    * Single-Threaded: simplifies order of operations and makes executing them really fast
        - the first request is always the first one served and all subsequent requests have to wait
        - you don't have to worry about complex locking mechanisms
            * don't have multiple threads to coordinate for access to a single resource
            * no deadlocks since no other threads or shared resources to wait on
        - avoids race conditions since operations only happen once at a time
    * In-Memory: makes operations very fast since it avoids the overhead of reading data from disk into memory through disk I/O
        - but we have a tradeoff of speed for durability, meaning data saved in memory will not persist
        - there are ways to minimze data loss but you don't get the same guarantees compared to a relational database
* fundamental data structures supported by Redis:
    - strings
    - hashes (objects)
    - lists
    - sets
    - sorted sets (priority queues)
    - bloom filters
    - geospatial indexes
    - time series
* also supports different communication patterns like Pub/Sub and Streams
* __core structure underneath Redis is a key-value store__
    - keys = strings
    - values = any data structures supported by Redis
    - __the way you organize the keys will be the way your organize your data and scale your Redis cluster__

## Infrastructure Configurations

* can be run as a single node, a single node with a high availability (HA) replica, or as a cluster
* for clusters, the Redis clients have a set of "hash slots"
    - these hash slots map keys to a specific node
    - e.g. if node1 has keys [0,100] and you want to look up info on key 50, you know that node1 would have it
* gossip protocol: each node in a cluster is aware of other nodes
    - if you send a request to the wrong node to lookup a key, that node will tell you the correct node to query
* Redis clusters are pretty basic and thus are limited
    - e.g. Redis expects all data for a given request to be on a single node
    - reason being, you cannot have multi-key operations across multiple hash slots
    - since hash slots map to nodes, if you try to work with multiple keys, you cannot guarantee that those keys will belong to the same hash slot and thus the same node by extension
    - this is only a limitation in cluster-mode. without clustering, all keys would be on the same node, and thus would support multi-key operations
    - [Why are multi-key operations not supported on Redis cluster?](https://groups.google.com/g/redis-db/c/un6KAUaMDLI)

## Performance

* Redis is very fast
* O(100k) writes per second
* read latency = microsecond range
* since it's so fast, some anti-patterns in other database systems is feasible with Redis
    - e.g. really bad: 100 SQL requests to generate a list of items vs writing 1 SQL query for all the data
    - with Redis, this is still a terrible idea but with much less overhead

## Capabilities

### Redis as a Cache:

* most common use for Redis
* basically just use it as a regular key-value store
    - the root keys literally maps to a value and not to a data structure containing your value
        * i.e. the root key is your lookup key
    - can distribute this hash map across nodes in a cluster pretty easily since keys map to hash slots which map to nodes
        * would just have to rebalance the cluster
        * [Hash Slot Resharding and Rebalancing for Redis Cluster](https://severalnines.com/blog/hash-slot-resharding-and-rebalancing-redis-cluster/)
* each key would have a TTL (Time-To-Live)
    - Redis guarantees you'll never read a value of a key after the TTL has expired
    - TTL used as an eviction policy
    - depending on the eviction policy, if you want to create more entries in the cache but are running out of memory, Redis would evict entries with TTLs that are the closest to expiring
        * "volatile-ttl: Evict keys with the expire field set to true that have the shortest __remaining__ time-to-live (TTL) value."
* __doesn't solve the Hot Key Problem though__

### Redis as a Distributed Lock:

* a distributed lock controls access to a shared resource like a mutex but for distributed systems
* an example:
    - simple distributed lock with timeout
    - when we want to get the lock, we run the `INCR` command
    - if response = 1, we own the lock
    - if response > 1, have to wait and retry again until lock is released
    - when someone is done with the lock, we can `DEL` the keys so others can use it
* more sophisticated locks in Redis can use the Redlock Algorithm

### Redis for Leaderboards:

* we can make use of Redis' __SortedSets__ which we can query in O(logn) time
    - for scaling, Redis is better than SQL Databases because of its high write throughput and low read latency
    - Redis is in-memory so it's much faster than reading from disk
    - Redis is single-threaded so don't have to worry about locks
    - Also doesn't have to parse queries, update indexes, handle joins and transactions, etc.
* an example:
    - want to find posts which contain  keyword (e.g. "tiger") which has the most likes
    - can use SortedSets to maintain this list of words by adding the tweet and number of likes into it
    
### Redis for Rate Limiting:

* common algorithm for Rate Limiting: Fixed-Window Rate Limiter
    - i.e. # of requests does not exceed N over some fixed window of time W
* create a key for our rate limiter
    - value = # of requests so far
    - when request comes in, we increment that value
    - if the response > N, we wait
    - otherwise, we can make the request
    - we `EXPIRE` the key after the time period, W, so that the value is reset

### Redis for Proximity Search

* Redis natively supports geospatial indexes
* the search command, GEOSEARCH, runs in O(n + log(m)) time where n = # of elements in the radius and m = # of items inside the shape


### Redis for Event Sourcing

*  Redis stream = append-only logs
    - want to use this to add items to a log
    - then have some distributed mechanism to consume items from these logs
* an example:
    - want to have an asynchronous work queue
    - add items to these queue to have them processed
    - we have workers that work on these items but sometimes they can fail
    - to solve this, we can use a Consumer Group (CG)
    - the CG attaches to the stream and keeps track of the item we're currently working on
    - it distributes that work to a Worker
    - if the worker fails, the CG will reclaim the item and give it to a new worker

### Redis for Pub/Sub

* Redis natively supports a publish/subscribe (Pub/Sub) messaging pattern
    - usefulfor chat systems, real-time notifications, or when we want to decouple message producers from message consumers
    - Redis pub/sub is also now shared which allows for scalability
* client subscribes to channel => receives any messages published to that channel
    - only as long as connection remains open
* pub/sub clients use a single connection to each node in the cluster (rather than a connection per channel)
    - i.e. # of connections = # of nodes in a cluster
    - don't need millions of connections if you have millions of channels!
* __Redis Pub/Sub is NOT DURABLE__
    - delivery = at most once
        * if subscriber is offline when the message is published, it won't receive it
    - use Kafka/RabbitMQ or Redis Streams when you need message persistence, delivery guarantees,or ability to replay missed messages

## Shortcomings and Remediations

### Hot Key Issues

* the Hot Key Issue refers to a situation where a small number of keys receives a disproportionately high amount of traffic compared to others
* an example:
    - have items in an ecommerce store
    - have lots of nodes and spread items evenly
    - one item starts getting a large amount of traffic
    - node with that item has way higher load than the rest of the nodes
    - server will start failing
* possible solutions:
    - add an in-memory cache in the clients so that we reduce number of requests to Redis
    - store the same data in multiple keys and randomize requests so they spread across the cluster
    - add read replica instances and dynamically scale these with load